# Test `output` module 

*Test output module against reference data.*

> **IMPORTANT:** `pytest-notebook` runs the notebook and `diff`'s every cell against the previously saved result. So, the result of the test depends on the `diff` output and indirectly on the assertion. Logging is disabled to avoid comparing unnecesary strings. Keep in mind that the functions `assert_frame_equal` and `assert_series_equal` returns nothing when objects are identical.
>
> You will need to export the path to the reference data directory as an environment variable named `CARSUS_REFDATA` before running this notebook.

---

In [1]:
import os
import pandas as pd
from pandas.testing import assert_frame_equal, assert_series_equal

In [2]:
import logging
logger = logging.disable(logging.CRITICAL)

In [3]:
GFALL_IONS = "H-Si"
CHIANTI_IONS = "H-He"
CMFGEN_IONS = "Si_I-II"

## Load reference data

In [4]:
CARSUS_REFDATA = os.environ["CARSUS_REFDATA"]

In [5]:
fname = f"test_data_ku_{GFALL_IONS}_ch_{CHIANTI_IONS}_cm_{CMFGEN_IONS}.h5"

In [6]:
refdata = pd.HDFStore(os.path.join(CARSUS_REFDATA, fname))

## Create new atomic data

In [7]:
from carsus.io.nist import NISTWeightsComp, NISTIonizationEnergies
from carsus.io.kurucz import GFALLReader
from carsus.io.zeta import KnoxLongZeta
from carsus.io.chianti_ import ChiantiReader
from carsus.io.cmfgen import CMFGENEnergyLevelsParser, CMFGENOscillatorStrengthsParser, CMFGENReader
from carsus.io.output import TARDISAtomData

atomic_weights = NISTWeightsComp()
ionization_energies = NISTIonizationEnergies(GFALL_IONS)
gfall_reader = GFALLReader(ions=GFALL_IONS)
chianti_reader = ChiantiReader(ions=CHIANTI_IONS, collisions=True, priority=20)
zeta_data = KnoxLongZeta()

si_0_lvl = CMFGENEnergyLevelsParser(os.path.join(CARSUS_REFDATA, 'cmfgen/energy_levels/SiI_OSC')).base
si_0_osc = CMFGENOscillatorStrengthsParser(os.path.join(CARSUS_REFDATA, 'cmfgen/energy_levels/SiI_OSC')).base
si_1_lvl = CMFGENEnergyLevelsParser(os.path.join(CARSUS_REFDATA,'cmfgen/energy_levels/si2_osc_kurucz')).base
si_1_osc = CMFGENOscillatorStrengthsParser(os.path.join(CARSUS_REFDATA,'cmfgen/energy_levels/si2_osc_kurucz')).base

cmfgen_data = {(14,0): {'levels': si_0_lvl, 'lines': si_0_osc},
               (14,1): {'levels': si_1_lvl, 'lines': si_1_osc},}

cmfgen_reader = CMFGENReader(cmfgen_data, priority=20)

 ChiantiPy version 0.8.4 
 found PyQt5 widgets
 using PyQt5 widgets


In [8]:
atom_data = TARDISAtomData(atomic_weights,
                            ionization_energies,
                            gfall_reader,
                            zeta_data,
                            chianti_reader,
                            cmfgen_reader)

In [9]:
atomic_weights = atom_data.atomic_weights.base.loc[1:14]  # H-Si

In [10]:
ionization_energies = atom_data.ionization_energies.base

In [11]:
ionization_energies_prepared = atom_data.ionization_energies_prepared

In [12]:
levels_all = atom_data._get_all_levels_data().drop(columns=["ds_id"])

In [13]:
levels = atom_data.levels.drop(columns=["ds_id"])

In [14]:
levels_prepared = atom_data.levels_prepared

In [15]:
lines_all = atom_data._get_all_lines_data().drop(columns=["ds_id"])

In [16]:
lines = atom_data.lines.drop(columns=["ds_id"])

In [17]:
lines_prepared = atom_data.lines_prepared

In [18]:
macro_atom = atom_data.macro_atom

In [19]:
macro_atom_prepared = atom_data.macro_atom_prepared

In [20]:
macro_atom_references = atom_data.macro_atom_references

In [21]:
macro_atom_references_prepared = atom_data.macro_atom_references_prepared

In [22]:
collisions = atom_data.collisions.drop(columns=["btemp", "bscups"])

In [23]:
collisions_prepared = atom_data.collisions_prepared

In [24]:
zeta_data = atom_data.zeta_data.base

## Assert 

In [25]:
assert_frame_equal(atomic_weights, refdata['atomic_weights'])

In [26]:
assert_series_equal(ionization_energies, refdata['ionization_energies'])

In [27]:
assert_series_equal(ionization_energies_prepared, refdata['ionization_energies_prepared'])

In [28]:
assert_frame_equal(levels_all, refdata['levels_all'])

In [29]:
assert_frame_equal(levels, refdata['levels'])

In [30]:
assert_frame_equal(levels_prepared, refdata['levels_prepared'])

In [31]:
assert_frame_equal(lines_all, refdata['lines_all'])

In [32]:
assert_frame_equal(lines, refdata['lines'])

In [33]:
assert_frame_equal(lines_prepared, refdata['lines_prepared'])

In [34]:
assert_frame_equal(macro_atom, refdata['macro_atom'])

In [35]:
assert_frame_equal(macro_atom_prepared, refdata['macro_atom_prepared'])

In [36]:
assert_frame_equal(macro_atom_references, refdata['macro_atom_references'])

In [37]:
assert_frame_equal(macro_atom_references_prepared, refdata['macro_atom_references_prepared'])

In [38]:
assert_frame_equal(collisions, refdata['collisions'])

In [39]:
assert_frame_equal(collisions_prepared, refdata['collisions_prepared'])

In [40]:
assert_frame_equal(zeta_data, refdata['zeta_data'])

In [41]:
refdata.close()